# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [108]:
import logging
import os
import io
import joblib
import json
import math
import requests
import pandas as pd
import datetime
import azureml.core
import azureml.automl

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

from inference.utils import align_outputs

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'automl-forecast'

experiment=Experiment(ws, experiment_name)

quick-starts-ws-199953
aml-quickstarts-199953
southcentralus
d4ad7261-832d-46b2-b093-22156001df5b


In [3]:
# Create or attach a compute cluster
cluster_name = "canina-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

A new compute target has been created.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
DATA_DIR = os.path.join(os.path.abspath(os.getcwd()), "ojdata")
LAST_WEEK = 138
NUM_WEEK_TEST = 3
time_column_name = "week_start"
grain_column_names = ["store", "brand"]
target = "move"

# The start datetime of the first week in the record
FIRST_WEEK_START = pd.to_datetime("1989-09-14 00:00:00")

df = pd.read_csv(os.path.join(DATA_DIR, "yx.csv"))
df = df.loc[df.week <= LAST_WEEK]

In [5]:
# Convert logarithm of the unit sales to unit sales
df["move"] = df["logmove"].apply(lambda x: round(math.exp(x)))

# Add timestamp column
df["week_start"] = df["week"].apply(lambda x: FIRST_WEEK_START + datetime.timedelta(days=(x - 1) * 7))

In [6]:
# Split data into training and test sets

def split_last_n_by_grain(df, n):
    """Group df by grain and split on last n rows for each group."""
    df_grouped = df.sort_values(time_column_name).groupby(
        grain_column_names, group_keys=False
    )
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail


train_df, test_df = split_last_n_by_grain(df, NUM_WEEK_TEST)
train_df.drop("logmove", axis=1, inplace=True)
train_df.reset_index(drop=True)
test_df.reset_index(drop=True)

local_data_paths = [
    os.path.join(DATA_DIR, "train_automl.csv"),
    os.path.join(DATA_DIR, "test_automl.csv")
]

train_df.to_csv(local_data_paths[0], index=None, header=True)
test_df.to_csv(local_data_paths[1], index=None, header=True)

In [7]:
# Upload data to datastore

ds = ws.get_default_datastore()
ds.upload_files(
    files=local_data_paths, 
    target_path="dataset/", 
    overwrite=True, 
    show_progress=False)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


$AZUREML_DATAREFERENCE_a748e7d8638242ceaf7bce8050132bf0

In [8]:
train_dataset = Dataset.Tabular.from_delimited_files(path=ds.path("dataset/train_automl.csv"))

In [9]:
# Visualize the first five lines of the training dataset

train_dataset.to_pandas_dataframe().head(5)

,store,brand,week,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,move,week_start
0,2,1,40,1,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1,0.0,37.992326,8256,1990-06-14
1,2,1,46,1,0.060469,0.060312,0.045156,0.046719,0.049531,0.047813,0.045781,0.027969,0.042969,0.042031,0.038984,0,0.0,30.126667,6144,1990-07-26
2,2,1,47,1,0.060469,0.060312,0.045156,0.046719,0.037344,0.053021,0.045781,0.041406,0.048125,0.032656,0.038984,0,0.0,30.000000,3840,1990-08-02
3,2,1,48,1,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0,0.0,29.950000,8000,1990-08-09
4,2,1,50,1,0.060469,0.060312,0.043594,0.031094,0.049531,0.053021,0.046648,0.041406,0.042344,0.032656,0.038203,0,0.0,29.920000,8896,1990-08-23


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# Forecast settings
forecast_settings = {
    "time_column_name": time_column_name,
    "grain_column_names": grain_column_names,
    "max_horizon": NUM_WEEK_TEST
}
# Automl settings 
automl_settings = {
    "experiment_timeout_hours" : 1.2,
    "primary_metric" : "normalized_mean_absolute_error",
    "n_cross_validations" : 3
}

# Automl config 
automl_config = AutoMLConfig(
    task="forecasting",
    debug_log="automl_errors.log",
    training_data=train_dataset,
    label_column_name=target,
    compute_target=compute_target,
    enable_early_stopping=True,
    verbosity=logging.INFO,
    **automl_settings,
    **forecast_settings
)

In [11]:
# Submit the experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-forecast,AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
remote_run.wait_for_completion(show_output=False)

{'runId': 'AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-07-01T06:35:20.880651Z',
 'endTimeUtc': '2022-07-01T08:02:58.217876Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 1 hour(s) 12 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'canina-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-forecast","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-199953","workspace_name":"quick-starts-ws-199953","region":"southcentralus","compute_target":"canina-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pip

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# Get the best model
best_run_automl, best_automl_model = remote_run.get_output()

In [15]:
# List all files of the best run
# We will use it to download the model, the environment file and the scoring file

best_run_automl.get_file_names()

['automl_driver.py',
 'explanation/0abe4901/eval_data_viz.interpret.json',
 'explanation/0abe4901/expected_values.interpret.json',
 'explanation/0abe4901/features.interpret.json',
 'explanation/0abe4901/global_names/0.interpret.json',
 'explanation/0abe4901/global_rank/0.interpret.json',
 'explanation/0abe4901/global_values/0.interpret.json',
 'explanation/0abe4901/local_importance_values.interpret.json',
 'explanation/0abe4901/rich_metadata.interpret.json',
 'explanation/0abe4901/visualization_dict.interpret.json',
 'explanation/0abe4901/ys_pred_viz.interpret.json',
 'explanation/8c1bf2f8/expected_values.interpret.json',
 'explanation/8c1bf2f8/features.interpret.json',
 'explanation/8c1bf2f8/global_names/0.interpret.json',
 'explanation/8c1bf2f8/global_rank/0.interpret.json',
 'explanation/8c1bf2f8/global_values/0.interpret.json',
 'explanation/8c1bf2f8/local_importance_values.interpret.json',
 'explanation/8c1bf2f8/rich_metadata.interpret.json',
 'explanation/8c1bf2f8/visualization_d

In [16]:
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

In [30]:
# Download the best AutoML model
joblib.dump(value = best_automl_model,
           filename = 'model.pkl')

['model.pkl']

In [24]:
run_metrics = best_run_automl.get_metrics()
for m, val in run_metrics.items():
    print(f"{m}: {val}")

mean_absolute_percentage_error: 95.58172663345624
mean_absolute_error: 9682.832564720467
explained_variance: 0.08800651717195869
root_mean_squared_log_error: 0.8695950502106419
median_absolute_error: 2263.019157560532
root_mean_squared_error: 31927.000301509164
normalized_root_mean_squared_log_error: 0.1945550504526623
normalized_median_absolute_error: 0.07499737912954695
spearman_correlation: 0.7565418928644784
normalized_root_mean_squared_error: 0.14071916098297987
normalized_mean_absolute_error: 0.10960286633862626
r2_score: 0.07548594845059264
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152_20/predicted_true
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152_20/residuals
forecast_table: aml://artifactId/ExperimentRun/dcid.AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152_20/forecast_table


In [25]:
best_run_automl.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":1}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 '_aml_system_codegen': 'completed',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152_ModelExplain',
 'model_explanation': 'True'}

In [26]:
print(best_automl_model.steps)

[('timeseriestransformer', TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(blocked_transformers=None, column_purposes=None, dataset_language=None, prediction_transform_type=None, transformer_params=None), force_time_index_features=None, freq='W-THU', grain_column_names=['store', 'brand'], group=None, lookback_features_removed=False, max_horizon=3, origin_time_colname='origin', pipeline=Pipeline(memory=None, steps=[('make_numeric_na_dummies', MissingDummiesTransformer(numerical_columns=['week', 'constant', 'price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11', 'deal', 'feat', 'profit'])), ('impute_na_numeric_datetime', TimeSeriesImputer(end=None, freq='W-THU', impute_by_horizon=False, input_column=['week', 'constant', 'price1', 'price2', 'price3', 'price4', 'price5', 'price6', 'price7', 'price8', 'price9', 'price10', 'price11', 'deal', 'feat', 'profit'], limit=None, limi

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
# Register the best AutoML model
#automl_model_name = best_run_automl.properties["model_name"]
model_reg = best_run_automl.register_model(model_path = 'outputs/model.pkl',
                                           model_name = 'forecast_oj',
                                           description = "AutoML model for forecasting.",
                                           tags = None,
                                           properties = {'MAPE': run_metrics['normalized_mean_absolute_error']})

In [34]:
model_reg

Model(workspace=Workspace.create(name='quick-starts-ws-199953', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-199953'), name=forecast_oj, id=forecast_oj:1, version=1, tags={}, properties={'MAPE': '0.10960286633862626'})

In [36]:
model_reg.download(target_dir='outputs', exist_ok=True)

'outputs/model.pkl'

In [38]:
print("Model ID", best_run_automl.id)

Model ID AutoML_6804d7d0-39fa-402c-b2ff-d1d0883d8152_20


In [39]:
# Downoad the file containing the environment details 
best_run_automl.download_file('outputs/conda_env_v_1_0_0.yml', 'runenv.yml')

# Download the scoring file
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [40]:
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'runenv.yml')

In [41]:
inference_config = InferenceConfig(entry_script = 'score.py',
                                   environment = myenv)

In [42]:
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description="AutoML model to forecast Orange Juice data"
)

In [43]:
service = Model.deploy(
    workspace=ws,
    name="automl-forecast-oj",
    models=[model_reg],
    inference_config=inference_config,
    deployment_config=aciconfig
)

In [44]:
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-01 10:03:30+00:00 Creating Container Registry if not exists..
2022-07-01 10:13:31+00:00 Registering the environment.
2022-07-01 10:13:32+00:00 Building image..
2022-07-01 10:33:49+00:00 Generating deployment configuration.
2022-07-01 10:33:50+00:00 Submitting deployment to compute.
2022-07-01 10:33:53+00:00 Checking the status of deployment automl-forecast-oj..
2022-07-01 10:37:20+00:00 Checking the status of inference endpoint automl-forecast-oj.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [45]:
print(service.state)

Healthy


# SKIP 

In [104]:
model = Model(ws,automl_model_name, version=1)

In [32]:
# Create a file containing the environment details
best_run_automl.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

In [105]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description="AutoML model to forecast Orange Juice data"
)

service = Model.deploy(workspace=ws,
             name="automl", 
             models=[model], 
             inference_config=inference_config, 
             deployment_config=aciconfig)

In [106]:
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-28 19:15:23+00:00 Creating Container Registry if not exists.
2022-06-28 19:15:23+00:00 Registering the environment.
2022-06-28 19:15:24+00:00 Generating deployment configuration..
2022-06-28 19:19:51+00:00 Submitting deployment to compute..
2022-06-28 19:20:19+00:00 Checking the status of deployment automl..
2022-06-28 19:25:10+00:00 Checking the status of inference endpoint automl.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [107]:
print(service.state)

Healthy


# SKIP

TODO: In the cell below, send a request to the web service you deployed to test it.

In [46]:
# Prepare data to test it on the web service
X = test_df
ground_truth = test_df.pop(target).values

In [112]:
# Take 6 samples from all the test dataset
test_X = X.iloc[:6,:]

In [113]:
test_X = test_X.drop("logmove", axis=1)

In [114]:
# Visualize the small test data
test_X

,store,brand,week,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,week_start
85,2,1,136,1,0.05,0.05,0.05,0.05,0.04,0.05,0.03,0.04,0.03,0.02,0.03,0,0.00,33.54,1992-04-16
86,2,1,137,1,0.04,0.05,0.05,0.04,0.03,0.04,0.03,0.04,0.04,0.02,0.03,0,0.00,20.43,1992-04-23
87,2,1,138,1,0.04,0.04,0.05,0.04,0.04,0.05,0.04,0.04,0.04,0.03,0.03,1,1.00,11.29,1992-04-30
195,2,2,136,1,0.05,0.05,0.05,0.05,0.04,0.05,0.03,0.04,0.03,0.02,0.03,1,0.00,27.13,1992-04-16
196,2,2,137,1,0.04,0.05,0.05,0.04,0.03,0.04,0.03,0.04,0.04,0.02,0.03,0,0.00,33.30,1992-04-23
197,2,2,138,1,0.04,0.04,0.05,0.04,0.04,0.05,0.04,0.04,0.04,0.03,0.03,1,1.00,9.43,1992-04-30


In [97]:
def transform_data(date):
    return datetime.strftime(date,'%Y-%m-%dT00:00:00Z')

for i in range(len(test_X)):
    test_X['week_start'].iloc[i] = transform_data(test_X['week_start'].iloc[i])

In [116]:
req_data

'{"data": [{"store": 2, "brand": 1, "week": 136, "constant": 1, "price1": 0.04984375, "price2": 0.0476273634, "price3": 0.04765625, "price4": 0.0478125, "price5": 0.04359375, "price6": 0.0509375, "price7": 0.03109375, "price8": 0.03578125, "price9": 0.02640625, "price10": 0.02015625, "price11": 0.032421875, "deal": 0, "feat": 0.0, "profit": 33.54, "week_start": "1992-04-16T00:00:00Z"}, {"store": 2, "brand": 1, "week": 137, "constant": 1, "price1": 0.0416446872, "price2": 0.0519791667, "price3": 0.04765625, "price4": 0.0388009736, "price5": 0.03265625, "price6": 0.038125, "price7": 0.0328611622, "price8": 0.03609375, "price9": 0.03734375, "price10": 0.0221875, "price11": 0.032421875, "deal": 0, "feat": 0.0, "profit": 20.425098039215698, "week_start": "1992-04-23T00:00:00Z"}, {"store": 2, "brand": 1, "week": 138, "constant": 1, "price1": 0.03734375, "price2": 0.0389583333, "price3": 0.04765625, "price4": 0.03578125, "price5": 0.04359375, "price6": 0.0509375, "price7": 0.04203125, "price8

In [ ]:
req_data = json.dumps({'data':test_X.to_dict(orient='records')})

In [99]:
headers = {'Content-Type':'application/json'}

In [102]:
response = requests.post(service.scoring_uri, req_data, headers=headers)

In [105]:
print(response.text)

"{\"forecast\": [12808.533333333335, 12808.533333333335, 12808.533333333335, 8671.2, 8671.2, 8671.2], \"prediction_interval\": [\"[-49346.21513687965, 74963.2818035463]\", \"[-75091.55491913024, 100708.62158579691]\", \"[-94846.64894873949, 120463.71561540614]\", \"[-53483.548470212976, 70825.94847021297]\", \"[-79228.88825246358, 96571.28825246356]\", \"[-98983.98228207283, 116326.3822820728]\"], \"index\": [{\"week_start\": 703382400000, \"store\": 2, \"brand\": 1}, {\"week_start\": 703987200000, \"store\": 2, \"brand\": 1}, {\"week_start\": 704592000000, \"store\": 2, \"brand\": 1}, {\"week_start\": 703382400000, \"store\": 2, \"brand\": 2}, {\"week_start\": 703987200000, \"store\": 2, \"brand\": 2}, {\"week_start\": 704592000000, \"store\": 2, \"brand\": 2}]}"


In [111]:
pd.read_csv(io.StringIO(response.text)).columns

Index(['{\forecast\": [12808.533333333335', ' 12808.533333333335',
       ' 12808.533333333335.1', ' 8671.2', ' 8671.2.1', ' 8671.2]',
       ' \"prediction_interval\": [\"[-49346.21513687965',
       ' 74963.2818035463]\"', ' \"[-75091.55491913024',
       ' 100708.62158579691]\"', ' \"[-94846.64894873949',
       ' 120463.71561540614]\"', ' \"[-53483.548470212976',
       ' 70825.94847021297]\"', ' \"[-79228.88825246358',
       ' 96571.28825246356]\"', ' \"[-98983.98228207283',
       ' 116326.3822820728]\"]', ' \"index\": [{\"week_start\": 703382400000',
       ' \"store\": 2', ' \"brand\": 1}', ' {\"week_start\": 703987200000',
       ' \"store\": 2.1', ' \"brand\": 1}.1', ' {\"week_start\": 704592000000',
       ' \"store\": 2.2', ' \"brand\": 1}.2', ' {\"week_start\": 703382400000',
       ' \"store\": 2.3', ' \"brand\": 2}', ' {\"week_start\": 703987200000.1',
       ' \"store\": 2.4', ' \"brand\": 2}.1',
       ' {\"week_start\": 704592000000.1', ' \"store\": 2.5',
       ' \"

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Enable application insights
service.update(enable_app_insights=True)

In [106]:
# Print the logs of the Web service
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2022-07-01T10:37:09,860807000+00:00 - gunicorn/run 
2022-07-01T10:37:09,884924600+00:00 - iot-server/run 
/bin/bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2022-07-01T10:37:09,904129600+00:00 - rsyslog/run 
2022-07-01T10:37:09,905186300+00:00 | gunicorn/run | 
bash: /azureml-envs/azureml_7becf92bd9d8786204e7278bb441885c/lib/libtinfo.so.6: no version information available (required by bash)
2022-07-01T10:37:09,942056200+00:00 | gunicorn/run | ####################################

In [ ]:
# Remove the web service
service.delete()

# Remove the compute cluster
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
